In [2]:
import tensorflow as tf
from tensorflow import keras

from keras.models import Model
from keras.layers import Input, LSTM, Dense

import numpy as np

Code from https://keras.io/examples/nlp/lstm_seq2seq/

In [3]:
latent_dim = 256

In [4]:
path = r"fra-eng\fra.txt"

with open(path, "r", encoding = "utf-8") as f:
    lines = f.read().split("\n")

In [5]:
input_texts = []
output_texts = []
input_characters = set()
output_characters = set()

for line in lines[:10000]:
    
    input_text, output_text, _ = line.split("\t")

    output_text = '\t' + output_text + '\n'

    input_texts.append(input_text)
    output_texts.append(output_text)
        
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
                
    for char in output_text:
        if char not in output_characters:
            output_characters.add(char)

In [6]:
input_characters = sorted(list(input_characters))
output_characters = sorted(list(output_characters))

num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(output_characters)

max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in output_texts])

In [7]:
input_index = dict([(char,i) for i,char in enumerate(input_characters)])
output_index = dict([(char,i) for i,char in enumerate(output_characters)])

reverse_input_index = dict([(i, char) for i,char in enumerate(input_characters)])
reverse_output_index = dict([(i, char) for i,char in enumerate(output_characters)])

In [8]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
decoder_output_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

In [9]:
for i, (input_text, output_text) in enumerate(zip(input_texts, output_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_index[char]] = 1.0
    encoder_input_data[i, t + 1 :, input_index[" "]] = 1.0
    
    for t, char in enumerate(output_text):
        decoder_input_data[i, t, output_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_output_data[i, t - 1, output_index[char]] = 1.0
            
    decoder_input_data[i, t + 1 :, output_index[" "]] = 1.0
    decoder_output_data[i, t:, output_index[" "]] = 1.0

In [10]:
# Define an input sequence and process it.
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
encoder = keras.layers.LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [11]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, 71)]   0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None, 93)]   0                                            
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, 256), (None, 335872      input_1[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, None, 256),  358400      input_2[0][0]                    
                                                                 lstm[0][1]                   

In [12]:
model.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_output_data,
    batch_size=64,
    epochs=100,
    validation_split=0.2,
)
# Save model
model.save("TextTransaltionModel")

Epoch 1/100
125/125 [==============================] - 28s 193ms/step - loss: 1.4636 - accuracy: 0.7043 - val_loss: 1.0840 - val_accuracy: 0.7133
Epoch 2/100
125/125 [==============================] - 21s 167ms/step - loss: 0.8707 - accuracy: 0.7648 - val_loss: 0.8436 - val_accuracy: 0.7648
Epoch 3/100
125/125 [==============================] - 21s 170ms/step - loss: 0.7071 - accuracy: 0.8067 - val_loss: 0.7063 - val_accuracy: 0.7974
Epoch 4/100
125/125 [==============================] - 23s 184ms/step - loss: 0.5940 - accuracy: 0.8275 - val_loss: 0.6355 - val_accuracy: 0.8140
Epoch 5/100
125/125 [==============================] - 23s 183ms/step - loss: 0.5351 - accuracy: 0.8427 - val_loss: 0.5950 - val_accuracy: 0.8274
Epoch 6/100
125/125 [==============================] - 24s 196ms/step - loss: 0.4979 - accuracy: 0.8543 - val_loss: 0.5611 - val_accuracy: 0.8359
Epoch 7/100
125/125 [==============================] - 25s 202ms/step - loss: 0.4687 - accuracy: 0.8618 - val_loss: 0.5388 -

Epoch 57/100
125/125 [==============================] - 25s 198ms/step - loss: 0.0939 - accuracy: 0.9706 - val_loss: 0.5830 - val_accuracy: 0.8741
Epoch 58/100
125/125 [==============================] - 23s 187ms/step - loss: 0.0917 - accuracy: 0.9714 - val_loss: 0.5830 - val_accuracy: 0.8734
Epoch 59/100
125/125 [==============================] - 23s 187ms/step - loss: 0.0886 - accuracy: 0.9720 - val_loss: 0.5902 - val_accuracy: 0.8741
Epoch 60/100
125/125 [==============================] - 24s 193ms/step - loss: 0.0881 - accuracy: 0.9723 - val_loss: 0.5960 - val_accuracy: 0.8740
Epoch 61/100
125/125 [==============================] - 25s 196ms/step - loss: 0.0857 - accuracy: 0.9731 - val_loss: 0.5944 - val_accuracy: 0.8745
Epoch 62/100
125/125 [==============================] - 23s 187ms/step - loss: 0.0837 - accuracy: 0.9736 - val_loss: 0.6056 - val_accuracy: 0.8741
Epoch 63/100
125/125 [==============================] - 23s 187ms/step - loss: 0.0807 - accuracy: 0.9744 - val_loss: 0

INFO:tensorflow:Assets written to: TextTransaltionModel\assets


INFO:tensorflow:Assets written to: TextTransaltionModel\assets


In [13]:
model = keras.models.load_model("TextTransaltionModel")

In [14]:
encoder_inputs = model.input[0]  # input_1
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  # lstm_1
encoder_states = [state_h_enc, state_c_enc]
encoder_model = keras.Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]  # input_2
decoder_state_input_h = keras.Input(shape=(latent_dim,), name="input3")
decoder_state_input_c = keras.Input(shape=(latent_dim,), name="input4")
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)

decoder_states = [state_h_dec, state_c_dec]

decoder_dense = model.layers[4]

decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

In [15]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, output_index["\t"]] = 1.0

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_output_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
        states_value = [h, c]
    return decoded_sentence

In [16]:
for seq_index in range(50):
    
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print("-")
    print("Input sentence:", input_texts[seq_index])
    print("Translated sentence:", decoded_sentence)

-
Input sentence: Go.
Translated sentence: Va !

-
Input sentence: Go.
Translated sentence: Va !

-
Input sentence: Go.
Translated sentence: Va !

-
Input sentence: Hi.
Translated sentence: Salut !

-
Input sentence: Hi.
Translated sentence: Salut !

-
Input sentence: Run!
Translated sentence: File !

-
Input sentence: Run!
Translated sentence: File !

-
Input sentence: Run!
Translated sentence: File !

-
Input sentence: Run!
Translated sentence: File !

-
Input sentence: Run!
Translated sentence: File !

-
Input sentence: Run!
Translated sentence: File !

-
Input sentence: Run!
Translated sentence: File !

-
Input sentence: Run!
Translated sentence: File !

-
Input sentence: Run.
Translated sentence: Cours !

-
Input sentence: Run.
Translated sentence: Cours !

-
Input sentence: Run.
Translated sentence: Cours !

-
Input sentence: Run.
Translated sentence: Cours !

-
Input sentence: Run.
Translated sentence: Cours !

-
Input sentence: Run.
Translated sentence: Cours !

-
Input sentenc